In [1]:
import pandas as pd
pd.set_option('display.max_row', 1500) # 데이터프레임 크기 조절
pd.set_option('display.max_columns', 100)
import numpy as np
import os
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import random
import unicodedata
import urllib.request
import urllib
import sys
import re
import math

In [2]:
cnt = int(input('크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt/10)
f_dir = input('사진을 저장할 폴더를 지정하세요(예: d:\data\) : ')
query_txt ='사진저장'

크롤링 할 건수는 몇건입니까?: 417
사진을 저장할 폴더를 지정하세요(예: d:\data\) : d:\data\


In [3]:
# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

sec_name = '품질인증업소'
os.makedirs(f_dir+s+'-'+query_txt+'-'+sec_name)
os.chdir(f_dir+s+'-'+query_txt+'-'+sec_name)

ff_dir=f_dir+s+'-'+query_txt+'-'+sec_name
ff_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.txt'
fc_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.csv'
fx_name=f_dir+s+'-'+query_txt+'-'+sec_name+'\\'+s+'-'+query_txt+'-'+sec_name+'.xls'

img_dir = ff_dir+"\\images"
os.makedirs(img_dir)
os.chdir(img_dir)

s_time = time.time( )

#Step 3. 검색어 입력한 후 검색하여 해당 장르로 이동하기
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
driver= webdriver.Chrome()

query_url='https://korean.visitkorea.or.kr/other/kq_list.do?otdid=456a84d1-84c4-11e8-8165-020027310001&choiceTag=%ED%92%88%EC%A7%88%EC%9D%B8%EC%A6%9D%EC%88%99%EC%86%8C&choiceTagId=a57ff771-dc1d-11e8-8a40-02001c6b0001'
driver.get(query_url)
time.sleep(3)

# Access Denied 메시지가 나오면 아래코드로 쿠키를 삭제한다
driver.delete_all_cookies()
time.sleep(2)


In [4]:
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(1)

scroll_down(driver)

ranking2=[]
title2=[]
sim_address2=[]
tel2=[]
url2=[]

count = 1
file_no = 1

#Step 6. 내용을 수집합니다
print("\n")
print("===== 곧 수집된 결과를 출력합니다 ^^ ===== ")
print("\n")

time.sleep(2)

for x in range(1,page_cnt + 1) :
    
    scroll_down(driver)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    #이미지 추출하기
    img_src2=[]   # 이미지 URL 저장변수
    
    item_result = soup.find('ul','list_thumType flnon')
    item_result2 = item_result.find_all('li', 'bdr_nor')
    image_list = item_result.find_all('img')
    img_cnt = 0
    
    for li in item_result2:

            if count > cnt :
                break

            try :
                photo = image_list[img_cnt]['src']
                img_cnt += 1
            except AttributeError :
                continue
            
            full_photo = photo
#             full_photo = 'https:' + photo
            urllib.request.urlretrieve(full_photo,str(file_no)+'.jpg')
            time.sleep(0.5)

            if  file_no > cnt :
                break

            f = open(ff_name, 'a',encoding='UTF-8')
            f.write("-----------------------------------------------------"+"\n")
            print("-" *70)
            
            ranking = count
            print("1.목록번호:",ranking)
            f.write('1.목록번호:'+ str(ranking) + "\n")

            try :
                t = li.find('div',class_='tit').find('a').get_text().replace("\n","")
            except  :
                title = '업소이름가 없습니다'
                print(title.replace("\n",""))
                f.write('2.업소이름:'+ title + "\n")
            else :
                title = t.replace("\n","").strip()
                print("2.업소이름:", title.replace("\n","").strip())                  
                f.write('2.업소이름:'+ title + "\n")       
                              
            try :
                driver.find_element_by_xpath(f'//*[@id="contents"]/div[2]/div[1]/ul/li[{li}]/div[2]/div/a').click()
                html_1 = driver.page_source
                soup_1 = BeautifulSoup(html_1, 'html.parser')    
                sep_li = soup_1.find('div', 'area_txtView bottom').find_all('li')
                for i in sep_li:
                    x = i.find('strong')
                    y = x.get_text()
                    if y == '주소':
                        sim_address = i.find('span').get_text()
                        break
                driver.back()
            except  :
                sim_address = 'nothing'
                print("3.업소위치:", sim_address.replace("\n",""))
                f.write('3.업소위치:'+ sim_address + "\n")
            else :
                print("3.업소위치:", sim_address.replace("\n",""))
                f.write('3.업소위치:'+ sim_address + "\n")
            
            try :
                tel = li.find('div', 'area_txt').find_all('p')[1].get_text()
            except  :
                tel = 'nothing'
                print("4.업소전화:", tel.replace("\n",""))
                f.write('4.업소전화:'+ tel + "\n")
            else :
                print("4.업소전화:", tel.replace("\n",""))
                f.write('4.업소전화:'+ tel + "\n")
            
            try :
                y = li.find('div', 'area_txt').find('div', 'tit').find('a')['onclick']
                z = y.split('(')
                a = z[1].split(',')
                url_sep=[]
                print(url_sep)
                for i in a:
                    j = i.replace('"', '')
                    j = j.replace(')', '')
                    url_sep.append(j)
                url_1 = 'https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=' + url_sep[0] + '&big_category=' + url_sep[2] + '&mid_category' + url_sep[3] + '&big_area' + url_sep[4]
            except  :
                url = 'nothing'
                print("5.업소url:", url_1.replace("\n",""))
                f.write('5.업소url:'+ url_1 + "\n")
            else :
                print("5.업소url:", url_1.replace("\n",""))
                f.write('5.업소url:'+ url_1 + "\n")
            print("-" *70)

            f.close( )

            time.sleep(0.6)

            ranking2.append(ranking)
            title2.append(title.replace("\n",""))
            sim_address2.append(sim_address.replace("\n",""))
            tel2.append(tel.replace("\n",""))
            url2.append(url_1.replace("\n", ""))
                
            count += 1
            file_no += 1
          
    driver.delete_all_cookies()
    time.sleep(2)
    
    try:
        next_page_xpath = '//*[@id="' + str(x+1) + '"]'
        if (x%5 == 0):
            driver.find_element_by_xpath(next_page_xpath).click( )
        else:
            driver.find_element_by_link_text('%s' %(x+1)).click() # 다음 페이지번호 클릭
    except:
        pass

#step 9. 엑셀 형태로 저장하기

hotel_list = pd.DataFrame()
hotel_list['목록index']=ranking2
hotel_list['업소사진']=np.nan
hotel_list['업소이름']=pd.Series(title2)
hotel_list['업소주소']=pd.Series(sim_address2)
hotel_list['업소전화']=pd.Series(tel2)
hotel_list['업소url']=pd.Series(url2)

# csv 형태로 저장하기
hotel_list.to_csv(fc_name,encoding="utf-8-sig",index=True)

# 엑셀 형태로 저장하기
hotel_list.to_excel(fx_name ,index=True)

e_time = time.time( )
t_time = e_time - s_time

# txt 파일에 크롤링 요약 정보 저장하기
orig_stdout = sys.stdout
f = open(ff_name, 'a',encoding='UTF-8')
sys.stdout = f

print("\n")
print("=" *50)
print("총 소요시간은 %s 초 이며," %t_time)
print("총 저장 건수는 %s 건 입니다 " %count)
print("=" *50)

sys.stdout = orig_stdout
f.close( )

#driver.close( )

count -= 1

print("\n") 
print("=" *80)
print("1.요청된 총 %s 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 %s 건입니다" %(cnt,count))
print("2.총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("3.파일 저장 완료: txt 파일명 : %s " %ff_name)
print("4.파일 저장 완료: csv 파일명 : %s " %fc_name)
print("5.파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)
print(" 잠시만 기다리시면 수집된 결과가 저장된 엑셀 파일이 자동으로 열립니다~~^^")
print("=" *80)

#이미지 삽입하기

import win32com.client as win32   #pywin32 , pypiwin32 설치후 동작
import win32api  #파이썬 프롬프트를 관리자 권한으로 실행해야 에러없음
                 #파이썬 쉘을 관리자 권한으로 실행한 후 불러오기로 이 소스 실행하기

excel = win32.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(fx_name)
sheet = wb.ActiveSheet
sheet.Columns(3).ColumnWidth = 30   #  이미지 가로 사이즈에 맞게 컬럼 크기 조정
row_cnt = cnt+1
sheet.Rows("2:%s" %row_cnt).RowHeight = 120  #  이미지 세로 사이즈에 맞게 로우 크기 조정

ws = wb.Sheets("Sheet1")
col_name2=[]
file_name2=[]

for a in range(2,cnt+2) :
    col_name='C'+str(a)
    col_name2.append(col_name)
    time.sleep(0.3)

for b in range(1,cnt+1) :
    file_name=img_dir+'/'+str(b)+'.jpg'
    file_name2.append(file_name)
    time.sleep(0.3)

for i in range(0,cnt) :
    rng = ws.Range(col_name2[i])
    image = ws.Shapes.AddPicture(file_name2[i], False, True, rng.Left, rng.Top, 130, 100)
    excel.Visible=True
    excel.ActiveWorkbook.Save()
    time.sleep(0.3)



===== 곧 수집된 결과를 출력합니다 ^^ ===== 


----------------------------------------------------------------------
1.목록번호: 1
2.업소이름: 오이아호텔 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 051-261-0600
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=675c5120-f98d-4b0e-b735-1344c540a5ff&big_category=B02&mid_categoryB0201&big_area6
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 2
2.업소이름: 한옥마을 황토펜션[한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 010-9081-5411
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=732652f3-96de-4b12-be7e-7c2a78e0c482&big_category=B02&mid_categoryB0201&big_area31
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 3
2.업소이름: 비브릿지 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 033-372-2000
[]
5.업소url: https://korean.visitkorea.or

----------------------------------------------------------------------
1.목록번호: 23
2.업소이름: 케이스호텔((주)안동케이스) [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 054-857-0007
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=d75ed36a-5b86-43b4-a923-c14a32bbfd7f&big_category=B02&mid_categoryB0201&big_area35
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 24
2.업소이름: 라마다인천호텔 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 032-460-1100
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=4b7170c9-9efa-48b8-8487-59d2360ccb2f&big_category=B02&mid_categoryB0201&big_area2
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 25
2.업소이름: 라비드아틀란호텔 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 051-760-9400
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.

----------------------------------------------------------------------
1.목록번호: 45
2.업소이름: 한옥에서의하루예랑(미담전, 동심전) [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=3b91acdc-ab02-4443-82e3-000fdf62c5f9&big_category=B02&mid_categoryB0201&big_area31
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 46
2.업소이름: 솔호텔 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 063-261-7000
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=1e2ef291-8eaf-47f0-8a23-4ed127b7dbbf&big_category=B02&mid_categoryB0201&big_area37
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 47
2.업소이름: 아몬드호텔 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 051-469-1918
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=4c617

----------------------------------------------------------------------
1.목록번호: 67
2.업소이름: 왕의지밀 태조관 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 063-284-1004
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=ff1f95ff-3065-4ce0-bd8d-23447bd91b29&big_category=B02&mid_categoryB0201&big_area37
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 68
2.업소이름: 왕의지밀 문종관 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 063-284-1004
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=c9c447ce-2f5e-4bec-9eec-b0b2c7e426dc&big_category=B02&mid_categoryB0201&big_area37
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 69
2.업소이름: 산수동 한옥체험시설(여로) [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 062-227-8815
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail

----------------------------------------------------------------------
1.목록번호: 89
2.업소이름: 베스트웨스턴 해운대호텔[한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 051-664-1234
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=577f956d-6d28-4b18-a70e-ea6c6fa9d870&big_category=B02&mid_categoryB0201&big_area6
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 90
2.업소이름: 구 보성여관 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 061-858-7528
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=e8316528-b7c8-4efb-a791-c4b6b714f3ea&big_category=B02&mid_categoryB0201&big_area38
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 91
2.업소이름: 생비원[한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 010-6732-4578
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=9

...

----------------------------------------------------------------------
1.목록번호: 353
2.업소이름: 피카소호텔(강원)[한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 033-533-2500
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=2ddba13e-2f70-4fb7-b18d-0e2fa5b14eb9&big_category=B02&mid_categoryB0201&big_area32
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 354
2.업소이름: 이코노미호텔 인천부평점 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 032-529-4941
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=4b448027-104b-43b1-930d-e418bd79244a&big_category=B02&mid_categoryB0201&big_area2
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 355
2.업소이름: 포항전통문화체험관 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 054-280-9371
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detai

----------------------------------------------------------------------
1.목록번호: 375
2.업소이름: 소담정[한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 010-3540-7171
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=e578799a-4b19-44f3-ba97-92b085c7d027&big_category=B02&mid_categoryB0201&big_area35
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 376
2.업소이름: 경주한옥1번가[한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 010-9505-5367
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=74e4633d-7eea-4bd1-9965-bb7a33aa32cb&big_category=B02&mid_categoryB0201&big_area35
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 377
2.업소이름: 남원예촌 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 063-636-8001
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=6af

----------------------------------------------------------------------
1.목록번호: 397
2.업소이름: 계림궁[한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 054-776-8122
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=f6025acc-0380-4bf7-a56c-b500168ed4e2&big_category=B02&mid_categoryB0201&big_area35
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 398
2.업소이름: 가람초연재[한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 054-853-4828
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=e1fd96fa-f75b-44e6-80c3-4837c15c06ac&big_category=B02&mid_categoryB0201&big_area35
----------------------------------------------------------------------
----------------------------------------------------------------------
1.목록번호: 399
2.업소이름: 한채당 한옥체험관 [한국관광 품질인증/Korea Quality]
3.업소위치: nothing
4.업소전화: 031-792-8000
[]
5.업소url: https://korean.visitkorea.or.kr/detail/ms_detail.do?cotid=35

<ipython-input-4-dabba8dc48c4>:165: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  hotel_list.to_excel(fx_name ,index=True)




1.요청된 총 417 건의 리뷰 중에서 실제 크롤링 된 리뷰수는 417 건입니다
2.총 소요시간은 726.3 초 입니다 
3.파일 저장 완료: txt 파일명 : d:\data\2021-08-16-16-21-39-사진저장-품질인증업소\2021-08-16-16-21-39-사진저장-품질인증업소.txt 
4.파일 저장 완료: csv 파일명 : d:\data\2021-08-16-16-21-39-사진저장-품질인증업소\2021-08-16-16-21-39-사진저장-품질인증업소.csv 
5.파일 저장 완료: xls 파일명 : d:\data\2021-08-16-16-21-39-사진저장-품질인증업소\2021-08-16-16-21-39-사진저장-품질인증업소.xls 
 잠시만 기다리시면 수집된 결과가 저장된 엑셀 파일이 자동으로 열립니다~~^^
